<a href="https://colab.research.google.com/github/Anu-joseph2629/github-slideshow/blob/main/Cardio_care.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# the following install is used to add pyspark library into google colab
!pip install pyspark 
# with the help of pyspark the spark tech is internally installed using the command in line #4
from pyspark.sql import SparkSession
#the feature engineering commands are initiated using the lib in line #6
#feature eningeering is nothing but EDA(exploratory data analysis),by selecting apropriate coloums for prediction
from pyspark.ml.feature import VectorAssembler,StringIndexer
#classification or regression algo is selected using the following command
#the programmar can select either logistic regression ir random forest regression or GBT (gradient boosted tree)
from pyspark.ml.classification import RandomForestClassifier
#evaluation lib is used to chevck the accuracy of the model 
from pyspark.ml.evaluation import BinaryClassificationEvaluator


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
spark_cardio = SparkSession \
.builder \
.config("spark.executor.memory","70g") \
.config("spark.driver.memory","50g") \
.config("spark.memory.offHeap.enabled",True) \
.config("spark.memory.offHeap.size","16g") \
.config("spark.kryoserializer.buffer.max","400M") \
.config("spark.serializer","org.apache.spark.serializer.KryoSerializer") \
.appName("Cardio_Care") \
.getOrCreate()

cardio_df = spark_cardio.read.csv("/content/drive/MyDrive/PROJECT _CARDIO_CARE/heart_disease.csv" , header=True , inferSchema = True )
print("original heart data frame...")
cardio_df.show(5,False)
print("the schema of dataframe...")
cardio_df.printSchema()
# Total number of patients in DataFrame
print("Total Rows = ", cardio_df.count())




original heart data frame...
+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+-----+---------+-------+----------+
|male|age|education|currentSmoker|cigsPerDay|BPMeds|prevalentStroke|prevalentHyp|diabetes|totChol|sysBP|diaBP|BMI  |heartRate|glucose|TenYearCHD|
+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+-----+---------+-------+----------+
|1   |39 |4        |0            |0         |0     |0              |0           |0       |195    |106.0|70.0 |26.97|80       |77     |0         |
|0   |46 |2        |0            |0         |0     |0              |0           |0       |250    |121.0|81.0 |28.73|95       |76     |0         |
|1   |48 |1        |1            |20        |0     |0              |0           |0       |245    |127.5|80.0 |25.34|75       |70     |0         |
|0   |61 |3        |1            |30        |0     |0              |1           |0       |225  

In [48]:
from pyspark.sql.types import IntegerType, DoubleType
#data pre processing starts from here
#check wtr any data is null
cardio_df.filter("male is NULL").count()
cardio_df.filter("age is NULL").count()
cardio_df.filter("currentSmoker is NULL").count()
cardio_df.filter("cigsPerDay is NULL").count()
cardio_df.filter("BPMeds is NULL").count()
cardio_df.filter("prevalentStroke is NULL").count()
cardio_df.filter("prevalentHyp is NULL").count()
cardio_df.filter("diabetes is NULL").count()
cardio_df.filter("totChol is NULL").count()
cardio_df.filter("sysBP is NULL").count()
cardio_df.filter("diaBP is NULL").count()
cardio_df.filter("BMI is NULL").count()
cardio_df.filter("heartRate is NULL").count()
cardio_df.filter("glucose is NULL").count()
cardio_df.filter("TenYearCHD is NULL").count()
#findings - the above coloumns are clean and no null or corrupt records found
cardio_df.describe().show()
#change datatype of cig per day from srting to integer as follows 

cardio_df_changed = cardio_df.withColumn('male', cardio_df["male"].cast(DoubleType())) \
.withColumn('age', cardio_df["age"].cast(DoubleType())) \
.withColumn('education', cardio_df["education"].cast(DoubleType())) \
.withColumn('currentSmoker', cardio_df["currentSmoker"].cast(DoubleType())) \
.withColumn('cigsPerDay', cardio_df["cigsPerDay"].cast(DoubleType())) \
.withColumn('BPMeds',cardio_df["BPMeds"].cast(DoubleType())) \
.withColumn('prevalentStroke',cardio_df["prevalentStroke"].cast(DoubleType())) \
.withColumn('prevalentHyp',cardio_df["prevalentHyp"].cast(DoubleType())) \
.withColumn('diabetes',cardio_df["diabetes"].cast(DoubleType())) \
.withColumn('totChol', cardio_df["totChol"].cast(DoubleType())) \
.withColumn('sysBP', cardio_df["sysBP"].cast(DoubleType())) \
.withColumn('diaBP', cardio_df["diaBP"].cast(DoubleType())) \
.withColumn('BMI',cardio_df["BMI"].cast(DoubleType())) \
.withColumn('heartRate',cardio_df["heartRate"].cast(DoubleType())) \
.withColumn('glucose',cardio_df["glucose"].cast(DoubleType())) \
.withColumn('TenYearCHD',cardio_df["TenYearCHD"].cast(DoubleType()))

#crossveerification 
print("datatype changed schemas of cardio_df")
cardio_df_changed.printSchema()
cardio_df_changed.describe().show()

# Rename Columns

cardio_df_column_renamed = cardio_df_changed.withColumnRenamed("male","Gender") \
    .withColumnRenamed("age","Age") \
    .withColumnRenamed("education","education") \
    .withColumnRenamed("currentSmoker","smoking_habbit") \
    .withColumnRenamed("cigsPerDay","Cigarettes_per_day") \
    .withColumnRenamed("BPMeds","BP_Medicine") \
    .withColumnRenamed("prevalentStroke","Stroke_History") \
    .withColumnRenamed("prevalentHyp","Hypertension_History") \
    .withColumnRenamed("diabetes","Diabetes") \
    .withColumnRenamed("totChol","Total_Cholesterol") \
    .withColumnRenamed("sysBP","Systolic_Pressure") \
    .withColumnRenamed("diaBP","Diastolic_Pressure") \
    .withColumnRenamed("BMI","Body_Mass_Index") \
    .withColumnRenamed("heartRate","Heart_Rate") \
    .withColumnRenamed("glucose","Blood_Sugar_Level") \
    .withColumnRenamed("TenYearCHD","Diagnosis") 

print("Schema changed DataFrame...")
cardio_df_column_renamed.printSchema()





+-------+-------------------+-----------------+------------------+------------------+------------------+--------------------+--------------------+-------------------+--------------------+------------------+------------------+------------------+-----------------+------------------+------------------+-------------------+
|summary|               male|              age|         education|     currentSmoker|        cigsPerDay|              BPMeds|     prevalentStroke|       prevalentHyp|            diabetes|           totChol|             sysBP|             diaBP|              BMI|         heartRate|           glucose|         TenYearCHD|
+-------+-------------------+-----------------+------------------+------------------+------------------+--------------------+--------------------+-------------------+--------------------+------------------+------------------+------------------+-----------------+------------------+------------------+-------------------+
|  count|              58933|        

In [49]:
from pyspark.ml.feature import VectorIndexer, StringIndexer
#select the relevant coloumns for accurate prediction.this is termed as feature engineering 

input_colms=["Gender","Age","Cigarettes_per_day","BP_Medicine","Stroke_History","Hypertension_History","Diabetes","Total_Cholesterol","Systolic_Pressure","Diastolic_Pressure","Body_Mass_Index","Heart_Rate","Blood_Sugar_Level"]

output_colm='features'

result_indexer = StringIndexer(inputCol="Diagnosis",outputCol="label")

#before creating a model the fundamental ETL-EXTRACT TRANSFORM AND LOAD operation must be performed
#for extracting the features and vectorisation the following assembler can be used

cardio_df_assembler = VectorAssembler(inputCols=input_colms,outputCol=output_colm).setHandleInvalid("skip")

#inorder to transform the above input coloumn features use the following transform tech

cardio_df_feature_df = cardio_df_assembler.transform(cardio_df_column_renamed)

#crossverification

cardio_df_feature_df.printSchema()
cardio_df_feature_df.show(5,False)

cardio_label_df = result_indexer.fit(cardio_df_feature_df).transform(cardio_df_feature_df)

cardio_label_df.printSchema()

cardio_label_df.show(5, False)




root
 |-- Gender: double (nullable = true)
 |-- Age: double (nullable = true)
 |-- education: double (nullable = true)
 |-- smoking_habbit: double (nullable = true)
 |-- Cigarettes_per_day: double (nullable = true)
 |-- BP_Medicine: double (nullable = true)
 |-- Stroke_History: double (nullable = true)
 |-- Hypertension_History: double (nullable = true)
 |-- Diabetes: double (nullable = true)
 |-- Total_Cholesterol: double (nullable = true)
 |-- Systolic_Pressure: double (nullable = true)
 |-- Diastolic_Pressure: double (nullable = true)
 |-- Body_Mass_Index: double (nullable = true)
 |-- Heart_Rate: double (nullable = true)
 |-- Blood_Sugar_Level: double (nullable = true)
 |-- Diagnosis: double (nullable = true)
 |-- features: vector (nullable = true)

+------+----+---------+--------------+------------------+-----------+--------------+--------------------+--------+-----------------+-----------------+------------------+---------------+----------+-----------------+---------+------------

In [53]:
#build random forest model 
#before creating rf model first step is split the given dataset for training and testing , usually 70% for training and 30% is used for testing 

training_data, test_data = cardio_label_df.randomSplit([0.8,0.2],seed=5043)

#train the random forest model 
classifier = RandomForestClassifier() \
.setImpurity("gini") \
.setMaxDepth(10) \
.setNumTrees(30) \
.setFeatureSubsetStrategy("auto") \
.setSeed(5043)

#model building starts here
model=classifier.fit(training_data) 
predictions = model.transform(test_data)
#check the accuracy of the model
evaluator = BinaryClassificationEvaluator().setLabelCol("label").setRawPredictionCol("prediction").setMetricName("areaUnderROC")
accuracy = evaluator.evaluate(predictions)
print ("ACCURACY  OF  THE  MODEL  USING BINARY ...")
print(accuracy*100) 

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator_multi =  MulticlassClassificationEvaluator(labelCol="label", predictionCol="predictions")

accuracy_multi = evaluator.evaluate(predictions)

print("Accuracy Multi = %s" % (accuracy_multi))
print("Test Error Multi = %s" % (1.0 - accuracy_multi))

# Save the above model in Google Drive

#model.save("/content/drive/MyDrive/model_cardio_care")


ACCURACY  OF  THE  MODEL  USING BINARY ...
69.88357843137256
Accuracy Multi = 0.6988357843137255
Test Error Multi = 0.3011642156862745


In [55]:
# Test the above model using New User Input Data

df1 = spark_cardio.createDataFrame(
    [
        (1.0,39.0,0.0,0.0,0.0,0.0,0.0,195.0,106.0,70.0,26.97,80.0,77.0), 
        (0.0,61.0,30.0,0.0,0.0,1.0,0.0,225.0,150.0,95.0,28.58,65.0,103.0),
        (1.0,75.0,20.0,1.0,0.0,1.0,1.0,213.0,250.0,150.0,32.1,82.0,240.0),
        (1.0,23.0,0.0,0.0,0.0,0.0,0.0,150.0,120.0,80.0,20.1,72.0,140.0)
        
    ],
    ["Gender","Age","Cigarettes_per_day","BP_Medicine","Stroke_History","Hypertension_History","Diabetes","Total_Cholesterol","Systolic_Pressure","Diastolic_Pressure","Body_Mass_Index","Heart_Rate","Blood_Sugar_Level"]  # add your column names here
)

#the given new user input data is 
print ("the given new user input data is")
df1.show(5,False)
df2 = cardio_df_assembler.transform(df1)

#predict wtr th epatient is affected or not by using the model

df3 = model.transform(df2)

print("predicted result =")

df3.show()




the given new user input data is
+------+----+------------------+-----------+--------------+--------------------+--------+-----------------+-----------------+------------------+---------------+----------+-----------------+
|Gender|Age |Cigarettes_per_day|BP_Medicine|Stroke_History|Hypertension_History|Diabetes|Total_Cholesterol|Systolic_Pressure|Diastolic_Pressure|Body_Mass_Index|Heart_Rate|Blood_Sugar_Level|
+------+----+------------------+-----------+--------------+--------------------+--------+-----------------+-----------------+------------------+---------------+----------+-----------------+
|1.0   |39.0|0.0               |0.0        |0.0           |0.0                 |0.0     |195.0            |106.0            |70.0              |26.97          |80.0      |77.0             |
|0.0   |61.0|30.0              |0.0        |0.0           |1.0                 |0.0     |225.0            |150.0            |95.0              |28.58          |65.0      |103.0            |
|1.0   |75.0|20.0

In [58]:
# Load the saved model - model_cardio_care

model_from_disk = model.load("/content/drive/MyDrive/model_cardio_care")

df1 = spark_cardio.createDataFrame(
    [
        (1.0,39.0,0.0,0.0,0.0,0.0,0.0,195.0,106.0,70.0,26.97,80.0,77.0), 
        (0.0,61.0,30.0,0.0,0.0,1.0,0.0,225.0,150.0,95.0,28.58,65.0,103.0),
        (1.0,75.0,20.0,1.0,0.0,1.0,1.0,213.0,250.0,150.0,32.1,82.0,240.0),
        (1.0,23.0,0.0,0.0,0.0,0.0,0.0,150.0,120.0,80.0,20.1,72.0,140.0)
        
    ],
    ["Gender","Age","Cigarettes_per_day","BP_Medicine","Stroke_History","Hypertension_History","Diabetes","Total_Cholesterol","Systolic_Pressure","Diastolic_Pressure","Body_Mass_Index","Heart_Rate","Blood_Sugar_Level"]  # add your column names here
)

#the given new user input data is 
print ("the given new user input data is")
df1.show(5,False)
df2 = cardio_df_assembler.transform(df1)

#predict wtr th epatient is affected or not by using the model

df3 = model_from_disk.transform(df2)

print("predicted result =")

df3.show()



the given new user input data is
+------+----+------------------+-----------+--------------+--------------------+--------+-----------------+-----------------+------------------+---------------+----------+-----------------+
|Gender|Age |Cigarettes_per_day|BP_Medicine|Stroke_History|Hypertension_History|Diabetes|Total_Cholesterol|Systolic_Pressure|Diastolic_Pressure|Body_Mass_Index|Heart_Rate|Blood_Sugar_Level|
+------+----+------------------+-----------+--------------+--------------------+--------+-----------------+-----------------+------------------+---------------+----------+-----------------+
|1.0   |39.0|0.0               |0.0        |0.0           |0.0                 |0.0     |195.0            |106.0            |70.0              |26.97          |80.0      |77.0             |
|0.0   |61.0|30.0              |0.0        |0.0           |1.0                 |0.0     |225.0            |150.0            |95.0              |28.58          |65.0      |103.0            |
|1.0   |75.0|20.0

In [74]:
# Instead of giving new user input directly, input the user data thru Excel 
# Load the saved model - model_cardio_care
#!pip install pyspark

model_from_disk = model.load("/content/drive/MyDrive/model_cardio_care")

df1_excel = spark_cardio.read.csv("/content/drive/MyDrive/PROJECT _CARDIO_CARE/user_input_excel.csv",header=True,inferSchema=True) 

#print("user_input_excel")
#df1_excel.show()

cardio_df_assembler = VectorAssembler(inputCols=input_colms,outputCol=output_colm).setHandleInvalid("skip")

df2_excel = cardio_df_assembler.transform(df1_excel)

df3_excel = model_from_disk.transform(df2_excel)

#print("Final  prediction  output...")
df3_excel.show()

# create or replace temp view creates a new table in memory by name prediction_view

df3_excel.createOrReplaceTempView("prediction_view")

#.collect() func will store the collected output into the prediction_result
prediction_result = spark_cardio.sql("select prediction from prediction_view limit 1").collect()

#print("Original Prediction Result..", prediction_result)

# Original Prediction Result.. [Row(prediction=0.0)]
# To remove the [Row(prediction=0.0)] and extract the actual value 0.0 use the 
# below command prediction_result[0].asDict()["prediction"]

prediction_result_value_only = prediction_result[0].asDict()["prediction"]

# The output will be, prediction_result_value_only=0.0 or 1.0 accordingly

#Cross verify the if conditions

# prediction_result_value_only = 1.0
# prediction_result_value_only = 2.0

# All if conditions below working fine 

from termcolor import colored

if  prediction_result_value_only == 0.0:
    print(colored("Patient Not Affected with Coronory Heart Disease", "yellow"))
elif prediction_result_value_only == 1.0:          
    print(colored("Patient Affected with Coronory Heart Disease", "green"))
else:
     print(colored("No predictions available for this Patient", "red"))




+------+---+------------------+-----------+--------------+--------------------+--------+-----------------+-----------------+------------------+---------------+----------+-----------------+--------------------+--------------------+--------------------+----------+
|Gender|Age|Cigarettes_per_day|BP_Medicine|Stroke_History|Hypertension_History|Diabetes|Total_Cholesterol|Systolic_Pressure|Diastolic_Pressure|Body_Mass_Index|Heart_Rate|Blood_Sugar_Level|            features|       rawPrediction|         probability|prediction|
+------+---+------------------+-----------+--------------+--------------------+--------+-----------------+-----------------+------------------+---------------+----------+-----------------+--------------------+--------------------+--------------------+----------+
|     0| 61|                25|          1|             0|                   1|       0|              160|              140|               120|             40|        82|              200|[0.0,61.0,25.0,1....|[1